# Answering Business Questions with SQL

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Total sold tracks per genre

In [3]:
%%sql
WITH usa_invoices AS 
                    (
                    SELECT  il.invoice_line_id,
                            il.quantity,
                            g.name genre,
                            i.billing_country
                    FROM invoice_line il 
                    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
                    INNER JOIN track t ON t.track_id = il.track_id
                    INNER JOIN genre g ON t.genre_id = g.genre_id
                    WHERE i.billing_country = "USA")
    
SELECT  usa_invoices.genre genre,
        SUM(usa_invoices.quantity) total_sold,
        CAST (SUM(usa_invoices.quantity) AS float)/(SELECT SUM(quantity) FROM usa_invoices) percentage
FROM usa_invoices 
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

Done.


genre,total_sold,percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


The chinook record stroe has signed a deal with a new label that produces different music genre. I have been tasked to suggest from which 3 of 4 artists the store should choose to buy albums. The 4 artists produce respectively Hip-Hop, Punk, Rock & Blues.
My recommendation is to buy the 3 albums with the genres : Blues, Pop & Hip Hop since they are the three best selling genres in the proposed list.

## Analyzing Employee Sales Performance 

In [4]:
%%sql
SELECT 
        e.first_name || " " || e.last_name employee_name,
        e.title,
        e.birthdate,
        e.hire_date,
        SUM(i.total) total_dollar_amount_of_sales
FROM invoice i
INNER JOIN customer c ON c.customer_id = i.customer_id
INNER JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY 1
ORDER BY 3 DESC;

Done.


employee_name,title,birthdate,hire_date,total_dollar_amount_of_sales
Jane Peacock,Sales Support Agent,1973-08-29 00:00:00,2017-04-01 00:00:00,1731.5100000000039
Steve Johnson,Sales Support Agent,1965-03-03 00:00:00,2017-10-17 00:00:00,1393.920000000002
Margaret Park,Sales Support Agent,1947-09-19 00:00:00,2017-05-03 00:00:00,1584.0000000000034


We can see that younger agents have higher total amount of sales. One reason for this could be that customer are generally young and a younger support agent can better communicate with them.

## Analyzing sales by country

In [5]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       i.*
     FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )
    
SELECT
    country,
    customers,
    total_sales,
    avg_sales_value_per_customer,
    avg_order_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(total) total_sales,
        SUM(total) / count(distinct customer_id) avg_sales_value_per_customer,
        SUM(total) / count(distinct invoice_id) avg_order_value,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,avg_sales_value_per_customer,avg_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


## Albums vs. individual tracks purchases

In [26]:
%%sql

WITH ita AS (
SELECT i.invoice_id ,
    t.track_id,
    a.album_id
FROM invoice i
INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN album a ON a.album_id = t.album_id )


SELECT  album_purchase,
        COUNT(invoice_id) number_of_invoices,
        CAST(COUNT(invoice_id) AS float) / (SELECT COUNT(*) FROM invoice) percentage

FROM

(  

SELECT i.invoice_id,
        CASE
            WHEN 
                (
                SELECT track_id 
                FROM (SELECT t.track_id FROM track t 
                      INNER JOIN album a ON a.album_id = t.album_id 
                      WHERE t.album_id = (SELECT ita.album_id FROM ita WHERE ita.invoice_id = i.invoice_id LIMIT 1)) 
                 
                EXCEPT
                
                SELECT track_id 
                FROM (SELECT t.track_id FROM track t 
                      INNER JOIN invoice_line il ON il.track_id = t.track_id
                      INNER JOIN invoice i2 ON i2.invoice_id = il.invoice_id
                      WHERE i2.invoice_id = i.invoice_id) 
                )
                IS NULL
            AND
                (
                SELECT track_id 
                FROM (SELECT t.track_id FROM track t 
                      INNER JOIN invoice_line il ON il.track_id = t.track_id
                      INNER JOIN invoice i2 ON i2.invoice_id = il.invoice_id
                      WHERE i2.invoice_id = i.invoice_id)
                
                EXCEPT
                
                SELECT track_id 
                FROM (SELECT t.track_id FROM track t 
                      INNER JOIN album a ON a.album_id = t.album_id 
                      WHERE t.album_id = (SELECT ita.album_id FROM ita WHERE ita.invoice_id = i.invoice_id LIMIT 1)) 
                ) 
                IS NULL
                THEN "yes"
                ELSE "no"
        END AS "album_purchase"
    
FROM invoice i 
    
)

GROUP BY 1;

Done.


album_purchase,number_of_invoices,percentage
no,500,0.8143322475570033
yes,114,0.18566775244299674


18 percent of the purchases are album purchases so I suggest to continue buying full albums from record companies

## Which artist is used in the most playlists 

In [30]:
%%sql

SELECT ar.name artist,
        COUNT(playlist_id) number_of_playlists

FROM playlist_track pt
INNER JOIN track t ON t.track_id = pt.track_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

Done.


artist,number_of_playlists
Iron Maiden,516
U2,333
Metallica,296
Led Zeppelin,252
Deep Purple,226


The Metal Band Iron Maiden are used the most in playlists

## How many tracks have been purchased vs not purchased?

In [46]:
%%sql

SELECT purchased_or_not,
        count(track_id) number_or_tracks
FROM
(
SELECT distinct t.track_id ,
        CASE 
            WHEN COUNT(il.invoice_line_id) = 0 THEN "not_purchased"
            ELSE "purchased"
        END AS "purchased_or_not"
        
FROM track t
LEFT JOIN invoice_line il ON il.track_id = t.track_id
GROUP BY 1
)
GROUP BY 1;

Done.


purchased_or_not,number_or_tracks
not_purchased,1697
purchased,1806


## Is the range of tracks in the store reflective of their sales popularity?

In [58]:
%%sql
WITH overall_invoices AS 
                    (
                    SELECT  il.invoice_line_id,
                            il.quantity,
                            g.name genre
                    FROM invoice_line il 
                    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
                    INNER JOIN track t ON t.track_id = il.track_id
                    INNER JOIN genre g ON t.genre_id = g.genre_id
                    )
    
SELECT  overall_invoices.genre genre,
        SUM(overall_invoices.quantity) total_sold,
        CAST (SUM(overall_invoices.quantity) AS float)/(SELECT SUM(quantity) FROM overall_invoices) percentage
FROM overall_invoices 
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

Done.


genre,total_sold,percentage
Rock,2635,0.553920538154299
Metal,619,0.13012402774858103
Alternative & Punk,492,0.10342652932520496
Latin,167,0.03510615934412445
R&B/Soul,159,0.033424427159974776


In [57]:
%%sql

SELECT  g.name genre,
        COUNT(t.track_id) total_tracks,
        CAST(COUNT(t.track_id) AS float) / (SELECT COUNT(track_id) FROM track) percentage
                            
FROM track t 
INNER JOIN genre g ON t.genre_id = g.genre_id
GROUP BY 1
ORDER BY 3 DESC
LIMIT 5;

Done.


genre,total_tracks,percentage
Rock,1297,0.3702540679417642
Latin,579,0.1652868969454753
Metal,374,0.10676562946046246
Alternative & Punk,332,0.09477590636597202
Jazz,130,0.03711104767342278


When we compare the range of tracks above with their sales popularity, we can say that they fit together.

## Do protected vs non-protected media types have an effect on popularity?

In [70]:
%%sql
SELECT protected_or_not,
        SUM(total_sales) total_sales
FROM
(
SELECT  t.track_id ,
        SUM(il.quantity) total_sales,
        CASE 
            WHEN mt.name LIKE "Protected%" THEN "protected"
            ELSE "non_protected"
        END AS "protected_or_not"     
FROM track t
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
LEFT JOIN invoice_line il ON il.track_id = t.track_id
GROUP BY 1
)
GROUP BY 1;

Done.


protected_or_not,total_sales
non_protected,4315
protected,442


Non protected media types make 10 times more sales